In [1]:
import glob
import os
import subprocess

import requests
from config import SETTINGS
from geo.Geoserver import Geoserver
from minio import Minio
from minio.error import InvalidResponseError
from tqdm.notebook import tqdm

# Loading GeoTiffs into GeoNode

This notebook illustrates how to load GeoTiffs into GeoServer/GeoNode using GDAL for data processing and the [GeoNode API](https://docs.geonode.org/en/master/devel/index.html). [Geoserver-Rest](https://geoserver-rest.readthedocs.io/en/latest/advanced_uses.html) may also be useful.

## 1. Files to process

The raw mosaics are not well optimised for display online. The first steps are therefore to convert them to Cloud Optimised Geotiffs (COGs), using either lossless or lossy compressions and internal tiling.

In [2]:
# Folder containing image mosaics
data_fold = r"/home/notebook/shared-seabee-ns9879k/all-mosaics"

# Output folder for Cloud Optimised Geotiffs (COGs)
cog_dir = r"/home/notebook/cogs/"

# Set compression type for COGs. If lossless = True, 'LZW' will be used.
# Otherwise, JPEG compression with QUALITY=50 and conversion to 8-bit
# will be used (this creates much smaller files, but also loses information).
lossless = False

In [3]:
# Get list of files to process
search_path = os.path.join(data_fold, "*.tif")
flist = glob.glob(search_path)

# Print file sizes
print("Raw mosaic sizes:")
for fpath in flist:
    fname = os.path.basename(fpath)
    fsize_mb = os.path.getsize(fpath) / 1e6
    print(f"{fname:<65}: {fsize_mb:.2f} MB")

Raw mosaic sizes:
Olberg_RGB_45m_Re_reprocess_dsm.tif                              : 721.52 MB
Olberg_RGB_45m_Re_reprocess_transparent_mosaic_group1.tif        : 553.21 MB
ne_akeroya_rgb_ms_5cm_compress_8bit.tif                          : 731.09 MB
akeroya_ms_10cm_8bit_compress.tif                                : 648.21 MB


## 2. Optimising files for display online

The blog post [here](http://blog.cleverelephant.ca/2015/02/geotiff-compression-for-dummies.html) from Paul Ramsey provides a useful staring point for optimising GeoTiffs. Note that, for very large files, the [GeoServer documentation](https://docs.geoserver.org/stable/en/user/tutorials/imagepyramid/imagepyramid.html) recommends building **external image pyramids** rather than **internal overview layers**. Some example bash scripts for building external pyramids are [here](https://www.ianturton.com/tutorials/bluemarble.html). However, loading COGs with internal overviews is easier to automate via the Python API and it's what Paul Ramsey recommends, so we'll test this first.

**Note:** Using `JPEG` compression (`lossless = False` above) will achieve the maximum reduction in file size. This is OK for visualisation online (e.g. for displaying datasets in GeoNode), but the resulting files **must not be used for machine learning**. JPEG compression is lossy and the JPEG format only supports 8-bit data (i.e. other bit depths must be converted). **For the ML we should always work with the original resolution files, saved with `LZW` compression**, which is lossless. However, using `JPEG` compression for visualisation substantially reduces file sizes, giving a smoother user experience.

In [4]:
cog_list = []
for fpath in flist:
    fname = os.path.basename(fpath)
    cog_path = os.path.join(cog_dir, os.path.splitext(fname)[0] + "_cog.tif")
    cog_list.append(cog_path)
    if lossless:
        # Use LZW
        cmd = [
            "gdal_translate",
            "-of",
            "COG",
            "-co",
            "COMPRESS=LZW",
            fpath,
            cog_path,
        ]
    else:
        # Use convert to 8-bit and JPEG with 50% quality
        cmd = [
            "gdal_translate",
            "-of",
            "COG",
            "-ot",
            "Byte",
            "-co",
            "COMPRESS=JPEG",
            "-co",
            "QUALITY=50",
            fpath,
            cog_path,
        ]
    subprocess.check_call(cmd)

Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.


Input file size is 17544, 23960
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 17544, 23960
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 31328, 17859
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26343, 16220
0...10...20...30...40...50...60...70...80...90...100 - done.


In [ ]:
# Get processed file sizes (including all overview layers etc.)
print("COG mosaic sizes:")
for cog_path in cog_list:
    fname = os.path.basename(cog_path)
    fsize_mb = os.path.getsize(cog_path) / 1e6
    print(f"{fname:<65}: {fsize_mb:.2f} MB")

The files sizes above using JPEG compression are *much* smaller than the originals (especially because the COGs include internally tiles overviews).

## 3. Upload COGs via the GeoNode API

In [ ]:
base_url = "https://geonode.seabee.sigma2.no/api/v2/uploads/upload"
headers = {"Authorization": f"Bearer {SETTINGS.GEONODE_TOKEN}"}
for fpath in tqdm(cog_list):
    fname = os.path.split(fpath)[1]
    files = [
        (
            "base_file",
            (
                fname,
                open(fpath, "rb"),
                "application/octet-stream",
            ),
        ),
    ]

    response = requests.request("POST", base_url, headers=headers, files=files)
    response.raise_for_status()

## 4. Add files using `geoserver-rest`

[Geoserver-Rest](https://geoserver-rest.readthedocs.io/en/latest/advanced_uses.html) offers direct access to workspaces on GeoServer and it's much nicer to use than the GeoNode API. An alternative and faster/more reliable approach for adding data to GeoNode is therefore to use `geoserver-rest`, as illustrated in the code below. However, to get new datasets to show up in GeoNode it is necessary to run either `geonode updatelayers` or `python manage.py updatelayers` from the GeoNode shell (sometimes called the Django shell). I *think* this can only be done programatically from within the GeoNode container (i.e. it's not possible to call this from the Hub), however, it looks as though it can be configured via the GeoNode admin panel under

    Home > Management Commands Over HTTP > Management command jobs

With a bit more research, this may be the best way of getting data into GeoNode.

In [ ]:
# # Authernticate with GeoServer
# geo = Geoserver(
#     "https://geonode.seabee.sigma2.no/geoserver",
#     username=SETTINGS.GEOSERVER_USER,
#     password=SETTINGS.GEOSERVER_PASSWORD,
# )

In [ ]:
# # Upload COGs to GeoServer
# workspace = "geonode"

# search_path = os.path.join(cog, "*.tif")
# flist = glob.glob(search_path)
# for fpath in flist:
#     fname = os.path.basename(fpath)
#     layer_name = os.path.splitext(fname)[0]

#     # Will overwrite layer if it exists
#     status = geo.create_coveragestore(
#         layer_name=layer_name, path=fpath, workspace=workspace
#     )
#     print(status)

## 5. Save COGs on MinIO

The final step is to transfer the COGs to MinIO for long-term storage.

In [ ]:
# Login to MinIO
sigma2_client = Minio(
    "storage.seabee.sigma2.no",
    access_key=SETTINGS.ACCESS_ID,
    secret_key=SETTINGS.SECRET_KEY,
)

In [ ]:
bucket = "all-mosaics"
found = sigma2_client.bucket_exists(bucket)
if not found:
    sigma2_client.make_bucket(bucket)

for fpath in tqdm(cog_list):
    fname = os.path.split(fpath)[1]
    try:
        with open(fpath, "rb") as file_data:
            file_stat = os.stat(fpath)
            sigma2_client.put_object(
                bucket, f"cloud-optimised/{fname}", file_data, file_stat.st_size
            )
    except InvalidResponseError as err:
        print(err)
    os.remove(fpath)